In [ ]:
from collections import defaultdict

import utils

from math import log,exp,sqrt,pi
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from scipy.special import erf
import types
import numpy as np
      
# MultiVariate Point process event
class MVPPEvent:
    def __init__(self, dim=0 , time=0, dim_name=None):
        self.dim = dim
        self.time = time
        self.dim_name = dim_name
    def __str__(self):
        s = str(self.dim)+"*"+str(self.time)
        if(not self.dim_name is None):
            s+= "*"+self.dim_name
        return s
    
    def __gt__(self,e2):
        return self.time > e2.time
    def __eq__(self,e2):
        return self.time == e2.time
    def __lt__(self,e2):
        return self.time < e2.time
    def __le__(self,e2):
        return self.time<=e2.time
    def __cmp__(self,e2):
        return self.time <= e2.time
    def __hash__(self):
        return hash(str(self.dim)+","+str(self.time))
    
class MVPPSequence:
    def __init__(self, lstEvents):
        assert all(lstEvents[i].time <= lstEvents[i+1].time for i in xrange(len(lstEvents)-1))
        self.lstEvents = lstEvents
        self.T = lstEvents[-1].time
        self.t_0 = lstEvents[0].time
        self.TT = self.detangle()
        
        
    def __str__(self):
        s = ""
        for e in self.lstEvents:
            s += str(e) +", "
        return s.rstrip(",")
    
    def detangle(self):
        
        TT = defaultdict(list)   
        for e in self.lstEvents:
            TT[e.dim].append(e.time)
            #self.T = max(self.T,e.time)
            #self.t_0 = min(self.t_0, e.time)
   
        return TT
    def count_dim(self):
        return len(self.TT.keys())
    
    def append(self,evt):
        self.lstEvents.append(evt)
        self.TT[evt.dim].append(evt.time)
        self.T = max(self.T,evt.time)

class MultiVariateHawkesProcessModel:
    
    def __init__(self,dim,l_0=None,A=None,trig_kernel=None):

        assert dim>0
        self.dim = dim
        self.l_0 = l_0
        self.A   = A
        self.sigma = np.ones(shape=(dim,dim))
        
        if(self.l_0 is None):
            self.l_0 = np.zeros(self.dim)
        if(self.A is None):
            self.A = np.zeros(shape=(self.dim,self.dim))
        
        if(trig_kernel is None):
            self.trig_kernel = [[ExponentialKernel(beta=(1.0/72))
                                 for x in range(self.dim)] for y in range(self.dim)] 
        self.num_kernel_params = dim*dim
        self.eps = 1e-5
        self.fit_kernel_params = False
        self.num_params = self.dim + self.dim*self.dim + self.num_kernel_params
        
        self.sequences = None
            
    # c is the index of dimension            
    def lamda_c(self,t,c,seq):
        
        #s = self.l_0[c] + self.eps #log safe
        sum_d = np.zeros(self.dim) # contrib of each dimension till t
        
        for e in seq.lstEvents:
            if(e.time < t):
                sum_d[e.dim] += exp(-(t-e.time)/self.sigma[e.dim][c])
            
        return self.l_0[c] + self.eps + np.dot(sum_d,self.A[:,c]), sum_d
    
         
    #returns a d dim vector           
    def lamda(self,t,seq):
        l = np.zeros(self.dim) + self.l_0
        for e in seq.lstEvents:
            if(e.time<t):
                for d in xrange(self.dim):
                    l[d] += self.A[e.dim][d] * exp( -(t - e.time)/self.sigma[e.dim][d])
            else:
                break
        return sum(l), l
                
    
    def lamda_ub(self,t,l_t, seq):
        '''
        v = np.arrange(t,t+l_t,1)
        lt = map(lambda x: self.lamda(x,seq)[0], v)
        return max(lt)
        '''
        t1 = time.time()
        l = np.zeros(self.dim) + self.l_0
        for e in seq.lstEvents:
            if(e.time<t):
                for d in xrange(self.dim):
                    l[d] += self.A[e.dim][d] * exp( -(t - e.time)/self.sigma[e.dim][d])
            else:
                break
        t2 = time.time()
        return sum(l)
    
    #def Lambda(self,t,seq):
        
    # Should be used to compute log likelihood of any generic kernel
    def neg_log_likelihood_general(self):
        C = len(self.sequences)
        ll = 0
        
        grad = np.zeros(self.num_params)
        p    = self.num_kernel_params
        grad_kernel = grad[0:p]
        grad_l0 = grad[p:p+self.dim]
        grad_A = np.reshape(grad[self.dim: (self.dim+1)*self.dim],(self.dim,self.dim))
        
        tmp_grad_kernel = [[np.zeros(self.trig_kernel[i][j].get_num_params())
                            for j in range(self.dim)] for i in range(self.dim)] 
        
        
        for seq in self.sequences:
            res = 0
            TT  = seq.TT
            m_T = seq.T
            dims = TT.keys()

            for m in dims:
                if(len(TT[m])==0): continue

                for t in TT[m]:
                    lmda_d_t,sum_d = self.lamda_c(t,m,seq)
                    res+=log(lmda_d_t)
                    grad_l0[m]+= 1.0/lmda_d_t
                    grad_A[:,m]+= sum_d/lmda_d_t
                    
                    for e in seq.lstEvents:
                        if(e.time < t):
                             tmp_grad_kernel[e.dim][m]+= self.A[e.dim][m]*self.trig_kernel[e.dim][m].kernel_grad(t-e.time)
                                
                    for e in seq.lstEvents:
                        if(e.time<t):
                            tmp_grad_kernel[e.dim][m] /=lmda_d_t
                    
                _sum =0       
                for n in dims:
                    for k in range(len(TT[n])):
                        G = self.trig_kernel[n][m].kernel_integral(m_T - TT[n][k])
                        _sum = _sum + (self.A[n][m])*G
                        
                        G_g = self.trig_kernel[n][m].kernel_integral_grad(m_T - TT[n][k])
                        grad_A[n][m] -= G
                        tmp_grad_kernel[n][m]-=G_g

                res = res - self.l_0[m] * m_T - _sum
                grad_l0[m] -= m_T
            ll += res
        p =0
        for i in range(self.dim):
            for j in range(self.dim):
                r = self.trig_kernel[i][j].get_num_params()
                grad_kernel[p:p+r] = tmp_grad_kernel[i][j]
                p+=r
        return 0-ll, 0-grad
    
    def grad(self):
        return self.neg_log_likelihood_general()[1]
    
    # Only Applicable for exponential kernel. Faster then the above general method.    
    def neg_log_likelihood_exp(self):
        C = len(self.sequences)
        ll = 0
        
        grad = np.zeros(self.num_params)
        p    = 0
        grad_kernel = None
        grad_l0 = None
        grad_A = None
        tmp_grad_kernel = None
        if(self.fit_kernel_params):
            p = self.num_kernel_params
            grad_kernel = grad[0:p]
            grad_l0 = grad[p:p+self.dim]
            grad_A = np.reshape(grad[self.dim: (self.dim+1)*self.dim],(self.dim,self.dim))

            tmp_grad_kernel = [[np.zeros(self.trig_kernel[i][j].get_num_params())
                                for j in range(self.dim)] for i in range(self.dim)] 
        else:
            grad_l0 = grad[p:p+self.dim]
            grad_A = np.reshape(grad[self.dim: (self.dim+1)*self.dim],(self.dim,self.dim))
        
        for seq in self.sequences:
            res = 0
            TT  = seq.TT
            m_T = seq.T
            dims = TT.keys()
            for m in dims :
                if(len(TT[m])==0): continue



                Rdiag    = np.zeros(len(TT[m]))
                RNonDiag = np.zeros(len(TT[m]))

                index    = map(int,np.zeros(self.dim))

                Rdiag[0] = 0

                
                for i in range(1,len(TT[m])):

                    Rdiag[i] = (1.0+Rdiag[i-1])*self.trig_kernel[m][m].kernel(TT[m][i] -TT[m][i-1])

                RNonDiag[0] = 0.0

                for n in dims:
                    if(n==m): continue
                    for k in range(len(TT[n])):
                        if(TT[n][k]<TT[m][0]):
                            
                            RNonDiag[0] += self.trig_kernel[n][m].kernel(TT[m][0]-TT[n][k])

                for i in range(1,len(TT[m])):

                    RNonDiag[i] = (RNonDiag[i-1])*self.trig_kernel[m][m].kernel(TT[m][i] - TT[m][i-1])

                    for n in dims:
                        if (m==n):
                            continue
                        for k in range(index[n],len(TT[n])):
                            if (TT[n][k] >= TT[m][i-1]):
                                if (TT[n][k] < TT[m][i]):

                                    RNonDiag[i] += self.trig_kernel[n][m].kernel(TT[m][i] - TT[n][k])
                                else:
                                    index[n] = k
                                    break

                for i in range(len(TT[m])):
                    s = self.l_0[m] +self.eps
                    for n in dims:
                        if(m==n):
                            s = s + self.A[n][m]*Rdiag[i]
                        else:
                            s = s + self.A[n][m]*RNonDiag[i]
                    
                    grad_l0[m] += 1/s
                    for n in dims:
                        if(m==n):
                            grad_A[n][m] += RDiag[i]/s
                        else:
                            grad_A[n][m] += RNonDiag[i]/s
                        
                    res = res + log(s)
                _sum     = 0.0
                for n in dims:
                    ls = 0.0
                    for k in range(len(TT[n])):
                        ls = ls + self.trig_kernel[n][m].kernel_integral(m_T-TT[n][k])
                    grad_A[n][m] -= ls
                    _sum += ls
                    

                res = res - self.l_0[m] * m_T -  (self.A[n][m])*_sum                           
                grad_l0[m]-=seq.T
                
            ll += res
        return 0-ll
    
    
    def opt_callback_set_params(self,x):
        p = 0
        if(self.fit_kernel_params):
            for i in range(self.dim):
                for j in range(self.dim):
                    c = self.trig_kernel[i][j].get_num_params()
                    self.trig_kernel[i][j].set_params_callback(x[p:p+c])
                    p+=c

        self.l_0  = x[p:p+self.dim]
        p        += self.dim

        self.A = np.reshape(x[p:],(self.dim,self.dim))
    
    def opt_callback_get_init_x(self):
        n = 0
        x0 = []
        if(self.fit_kernel_params):
            for i in range(self.dim):
                for j in range(self.dim):
                    x0.extend(self.trig_kernel[i][j].get_init_params())

        n = self.dim + self.dim*self.dim        
        x0.extend((1e-8)*np.ones(n))

        return np.array(x0)
    
    def opt_callback_get_bounds(self):
        bnds = []
        if(self.fit_kernel_params):
            for i in range(self.dim):
                for j in range(self.dim):
                    bnds.extend(self.trig_kernel[i][j].get_bounds())
        n = self.dim + self.dim*self.dim        
        bnds.extend([(1e-8,10.0)]*(n))
        return bnds
        
    def print_params(self):
        if(self.fit_kernel_params):
            for i in range(self.dim):
                for j in range(self.dim):
                    self.trig_kernel[i][j].print_params()
        print "l_0: " +str(self.l_0)
        print "A:" + str(self.A)
        
    def fit_mle_params(self,sequences,max_iters=1000,fit_kernel_params=False,tol=1e-18):
        self.sequences = sequences
        C = len(self.sequences)
        self.fit_kernel_params = fit_kernel_params
        self.f = 0
        def obj(x):
            self.opt_callback_set_params(x)
            ll,g = self.neg_log_likelihood_general()
            #print ll
            self.f+=1 
            print ll/C, self.f
            return ll/C
        
        def jac(x):
            self.opt_callback_set_params(x)
            g = self.grad()
            #print x.shape, g.shape
            return g/C
        
        bnds = self.opt_callback_get_bounds()
        x0   = self.opt_callback_get_init_x()
        
        res = optimize.minimize(obj, x0= x0,jac=False,method="L-BFGS-B",bounds=bnds,
                                tol=tol,options={"disp":True,"maxiter":max_iters,"maxfun":max_iters})
        
        self.opt_callback_set_params(res.x)
        
        
    '''
    def fit_mle_params(self,prefix=None,df_pd_train=None,dc=None,tol=1e-18,fit_kernel_params=True):
        mle_obj = mle.MaximumLikelihoodEstimator(model = self,prefix=prefix, df_pd_train=df_pd_train,
        dc=dc,tol=tol)
        self.fit_kernel_params = fit_kernel_params
        mle_obj.estimate()
    '''     
    def predictNextEventTime(self,seq,num_sim,step=1):
        ot = OgataThinning()
        t = 0
        for i in range(num_sim):
            event = ot.SimulateNext(self,1000,seq,step)
            t += event.time
        return float(t) / num_sim
    def predictNextKEvents(self,seq,k=5,num_sim=25,step=1):
        n = len(seq.lstEvents)
        import copy
        seq = MVPPSequence(copy.deepcopy(seq.lstEvents))
        
        for j in range(k):
            t = self.predictNextEventTime(seq,num_sim,step)
            s,l= self.lamda(t,seq)
            i = np.argmax(l)
            e = MVPPEvent(time=t,dim=i)
            print j,e
            seq.append(e)
        return seq.lstEvents[n:]
    
    #def predictItem(self,seq, t, num_sim):
        
    
    def plot(self,seq):
        
        def __plot(T,d,t_min,t_max,label):
            print label
            fig = plt.figure(figsize=(20,5))
            ax1 = fig.add_subplot(311)

            ax1.set_xlim([t_min, t_max])

            v = np.arange(t_min,t_max, 0.1)
            v = v[v<=t_max]
            lamdas = map(lambda t: self.lamda_c(t,d,seq)[0], v)

            a1, = ax1.plot(v,lamdas,label=label)

            ax3 = fig.add_subplot(312)
            ax3.set_xlim([t_min, t_max])
            for t_i in T:
                ax3.axvline(t_i)
            plt.legend(handles=[a1])
            plt.show()
            
        TT = seq.TT
        dim_id_name = dict()
        for e in seq.lstEvents:
            dim_id_name[e.dim] = e.dim_name
        t_0 = seq.t_0-100
        T   = seq.T+100
                         
        for d in range(self.dim):
            if(len(TT[d])>0):
                label = dim_id_name[d]
                __plot(TT[d],d,t_0,T,label)
    '''
    def check_stability(self):
        if(self.trig_kernel== self.exp)            
    '''       
